# taq_cross_response_transactions_test

#### Juan Camilo Henao Londono - 21.05.2019
#### AG Guhr - Universitaet Duisburg-Essen

In [1]:
# Modules

import numpy as np
import os
import pickle
from matplotlib import pyplot as plt
import multiprocessing as mp
from itertools import product

__tau__ = 1000
__case__ = 'wang' # 'juan' or 'wang'
__returns__ = 'ret' # 'ret' or 'log'

In [2]:
def taq_trade_signs_all_transactions_data(ticker, year, month, day, *, model=__case__):
    """
    Obtain the trade signs from the TAQ data. The trade signs are calculated
    using the equation (1) of https://arxiv.org/pdf/1603.01580.pdf.
    As the trades signs are not directly given by the TAQ data, they must be
    infered by the trades prices. For further calculations we use the whole
    time range from the opening of the market at 9h40 to the closing at 15h50
    in seconds (22200 seconds).
        :param ticker: string of the abbreviation of the stock to be analized
         (i.e. 'AAPL')
        :param year: string of the year to be analized (i.e '2016')
        :param month: string of the month to be analized (i.e '07')
        :param day: string of the day to be analized (i.e '07')
    """''

    # Load data
    
    function_name = taq_trade_signs_all_transactions_data.__name__
    
    print('TAQ data')
    print(function_name)
    print('Processing data for the stock ' + ticker + ' the ' + year
              + '.' + month + '.' + day)
    
    time_t, ask_t = pickle.load(open(
        '../TAQ_2008/TAQ_py/TAQ_{}_trades_{}{}{}.pickle'
        .format(ticker, year, month, day), 'rb'))

    # Reproducing S. Wang values. In her results the time interval for the
    # trade signs is [34801, 57000]
    if (model == 'juan'):
        condition = time_t != 57000
    elif (model == 'wang'):
        condition = time_t != 34800
        
    time_t = time_t[condition]
    ask_t = ask_t[condition]

    # All the trades must have a price different to zero
    assert not np.sum(ask_t == 0)

    # Trades identified using equation (1)
    identified_trades = np.zeros(len(time_t))
    identified_trades[-1] = 1

    # Implementation of equation (1). Sign of the price change between
    # consecutive trades

    for t_idx, t_val in enumerate(time_t):

        diff = ask_t[t_idx] - ask_t[t_idx - 1]

        if (diff):

            identified_trades[t_idx] = np.sign(diff)

        else:

            identified_trades[t_idx] = identified_trades[t_idx - 1]

    # All the identified trades must be different to zero
    assert not np.sum(identified_trades == 0)
    
    if (not os.path.isdir('../Cross_response_individual_stock/taq_data_{1}/{0}/'
                          .format(function_name + '_' + model, year))):

        try:

            os.mkdir('../Cross_response_individual_stock/taq_data_{1}/{0}/'.
                     format(function_name + '_' + model, year))
            print('Folder to save data created')

        except FileExistsError:

            print('Folder exists. The folder was not created')

    pickle.dump((time_t, identified_trades), open(
        '../Cross_response_individual_stock/taq_data_{2}/{0}/{0}_{2}{3}{4}_{1}.pickle'
        .format(function_name + '_' + model, ticker, year, month, day),
        'wb'))

    print('Data Saved')
    print()

    return (time_t, ask_t, identified_trades)

In [3]:
def taq_cross_response_transactions_data(ticker_i, ticker_j, year, month, day,
                                          *, mod=__returns__, model=__case__):
    """
    Obtain the cross response function using the midpoint log returns of
    ticker i and trade signs of ticker j during different time lags. The data
    is adjusted to use only the values each second. Return an array with the
    cross response function.
        :param ticker_i: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param ticker_j: string of the abbreviation of the trade sign stock to
         be analized (i.e. 'AAPL')
        :param year: string of the year to be analized (i.e '2016')
        :param month: string of the month to be analized (i.e '07')
        :param day: string of the day to be analized (i.e '07')
        :param mod='log': select the midpoint price return. 'ret' for midpoint
         price return and 'log' for midpoint price log return. Default 'log'
    """
    
    if (ticker_i == ticker_j):

        # Self-response

        return None
    
    else:
        
        function_name = taq_cross_response_transactions_data.__name__

        print('TAQ data')
        print(function_name)
        print('Processing data for the stock i ' + ticker_i + ' and stock j '
              + ticker_j + ' the ' + year + '.' + month + '.' + day)

        midpoint_i = pickle.load(open(''.join((
                    '../Cross_response_individual_stock/taq_data_{1}/taq_midpoint'
                    + '_full_time_data/taq_midpoint_full_time_data_midpoint_{1}{2}'
                    + '{3}_{0}.pickle').split())
                    .format(ticker_i, year, month, day), 'rb'))

        if (model == 'juan'):
            time_m = np.array(range(34800, 57000))
        elif (model == 'wang'):
            time_m = np.array(range(34801, 57001))    

        time_t, trade_sign_j = pickle.load(open("".join((
                    '../Cross_response_individual_stock/taq_data_{1}/taq_trade_signs'
                    + '_all_transactions_data_{4}/taq_trade_signs_all_transactions_data_{4}'
                    + '_{1}{2}{3}_{0}.pickle').split())
                    .format(ticker_j, year, month, day, model), 'rb'))

        assert not np.sum(trade_sign_j == 0)
        assert not np.sum(midpoint_i == 0)

        # Array of the average of each tau. 10^3 s used by Wang
        cross_response_tau = np.zeros(__tau__)

        # Calculating the midpoint log return and the cross response function

        midpoint_t = 0. * trade_sign_j

        for t_idx, t_val in enumerate(time_m):
            condition = time_t == t_val
            len_c = np.sum(condition)
            midpoint_t[condition] = midpoint_i[t_idx] * np.ones(len_c)

        assert not np.sum(midpoint_t == 0)

        # Depending on the tau value
        for tau_idx in range(__tau__):

            trade_sign_tau = 1 * trade_sign_j[1:-tau_idx-1]
            trade_sign_no_0_len = len(trade_sign_tau)
            # Obtain the midpoint log return. Displace the numerator tau
            # values to the right and compute the return

            # midpoint price log returns
            if (mod == 'log'):
                log_return_i_sec = np.log(midpoint_t[tau_idx + 1: -1]
                                          / midpoint_t[:-tau_idx - 2])

            # midpoint price returns
            elif (mod == 'ret'):

                log_return_i_sec = (midpoint_t[tau_idx + 1: -1]
                                    - midpoint_t[:-tau_idx - 2]) \
                    / midpoint_t[:-tau_idx - 2]

            # Obtain the cross response value
            if (trade_sign_no_0_len != 0):
                product = log_return_i_sec * trade_sign_tau
                cross_response_tau[tau_idx] = (np.sum(product)
                                               / trade_sign_no_0_len)

        if (not os.path.isdir('../Cross_response_individual_stock/taq_data_{1}/{0}/'
                              .format(function_name + '_' + mod + '_' + model, year))):

            try:

                os.mkdir('../Cross_response_individual_stock/taq_data_{1}/{0}/'.
                         format(function_name + '_' + mod + '_' + model, year))
                print('Folder to save data created')

            except FileExistsError:

                print('Folder exists. The folder was not created')

        pickle.dump(cross_response_tau, open(
            '../Cross_response_individual_stock/taq_data_{3}/{0}/{0}_{3}{4}{5}_{1}i_{2}j.pickle'
            .format(function_name + '_' + mod + '_' + model, ticker_i, ticker_j, year, month, day),
            'wb'))

        print('Data Saved')
        print()        


        return cross_response_tau

In [4]:
def taq_self_response_transactions_data(ticker, year, month, day,
                                        *, mod=__returns__, model=__case__):
    """
    Obtain the cross response function using the midpoint log returns of
    ticker i and trade signs of ticker j during different time lags. The data
    is adjusted to use only the values each second. Return an array with the
    cross response function.
        :param ticker_i: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param ticker_j: string of the abbreviation of the trade sign stock to
         be analized (i.e. 'AAPL')
        :param year: string of the year to be analized (i.e '2016')
        :param month: string of the month to be analized (i.e '07')
        :param day: string of the day to be analized (i.e '07')
        :param mod='log': select the midpoint price return. 'ret' for midpoint
         price return and 'log' for midpoint price log return. Default 'log'
    """
            
    function_name = taq_self_response_transactions_data.__name__

    print('TAQ data')
    print(function_name)
    print('Processing data for the stock ' + ticker + ' the ' +
          year + '.' + month + '.' + day)

    midpoint_i = pickle.load(open(''.join((
                '../Cross_response_individual_stock/taq_data_{1}/taq_midpoint'
                + '_full_time_data/taq_midpoint_full_time_data_midpoint_{1}{2}'
                + '{3}_{0}.pickle').split())
                .format(ticker, year, month, day), 'rb'))

    if (model == 'juan'):
        time_m = np.array(range(34800, 57000))
    elif (model == 'wang'):
        time_m = np.array(range(34801, 57001))    

    time_t, trade_sign_j = pickle.load(open("".join((
                '../Cross_response_individual_stock/taq_data_{1}/taq_trade_signs'
                + '_all_transactions_data_{4}/taq_trade_signs_all_transactions_data_{4}'
                + '_{1}{2}{3}_{0}.pickle').split())
                .format(ticker, year, month, day, model), 'rb'))

    assert not np.sum(trade_sign_j == 0)
    assert not np.sum(midpoint_i == 0)

    # Array of the average of each tau. 10^3 s used by Wang
    self_response_tau = np.zeros(__tau__)

    # Calculating the midpoint log return and the cross response function

    midpoint_t = 0. * trade_sign_j

    for t_idx, t_val in enumerate(time_m):
        condition = time_t == t_val
        len_c = np.sum(condition)
        midpoint_t[condition] = midpoint_i[t_idx] * np.ones(len_c)

    assert not np.sum(midpoint_t == 0)

    # Depending on the tau value
    for tau_idx in range(__tau__):

        trade_sign_tau = 1 * trade_sign_j[1:-tau_idx-1]
        trade_sign_no_0_len = len(trade_sign_tau)
        # Obtain the midpoint log return. Displace the numerator tau
        # values to the right and compute the return

        # midpoint price log returns
        if (mod == 'log'):
            log_return_i_sec = np.log(midpoint_t[tau_idx + 1: -1]
                                      / midpoint_t[:-tau_idx - 2])

        # midpoint price returns
        elif (mod == 'ret'):

            log_return_i_sec = (midpoint_t[tau_idx + 1: -1]
                                - midpoint_t[:-tau_idx - 2]) \
                / midpoint_t[:-tau_idx - 2]

        # Obtain the cross response value
        if (trade_sign_no_0_len != 0):
            product = log_return_i_sec * trade_sign_tau
            self_response_tau[tau_idx] = (np.sum(product)
                                           / trade_sign_no_0_len)

    if (not os.path.isdir('../Cross_response_individual_stock/taq_data_{1}/{0}/'
                          .format(function_name + '_' + mod + '_' + model, year))):

        try:

            os.mkdir('../Cross_response_individual_stock/taq_data_{1}/{0}/'.
                     format(function_name + '_' + mod + '_' + model, year))
            print('Folder to save data created')

        except FileExistsError:

            print('Folder exists. The folder was not created')

    pickle.dump(self_response_tau, open(
        '../Cross_response_individual_stock/taq_data_{2}/{0}/{0}_{2}{3}{4}_{1}.pickle'
        .format(function_name + '_' + mod + '_' + model, ticker, year, month, day),
        'wb'))

    print('Data Saved')
    print()        


    return self_response_tau

In [5]:
def months_days_list(folder_path, ticker, year):
    """
    Generate two lists with the string with the numbers of the months
    (from '01' to '12') and the days of the data to be analyzed.
    """
    days = []
    days_list = []
    months_list = []

    for i in range(1, 32):
        if (i < 10):
            days.append('0' + str(i))
        else:
            days.append(str(i))

    for m in range(1, 13):
        if (m < 10):
            months_list.append('0' + str(m))
        else:
            months_list.append(str(m))

    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    for month in months_list:
        days_month = []
        for d in days:
            for file in files:
                val_split = file.split('_')
                date = val_split[-1].split('.')[0]
                val = val_split[1] + val_split[2] + date
                if (val == '{}quotes{}{}{}'.format(ticker, year, month, d)):
                    days_month.append(d)
        days_list += [days_month]

    return(months_list, days_list)

In [6]:
folder_path = '../TAQ_2008/TAQ_py/'
tickers = ['AAPL', 'MSFT']
year = '2008'

(months_list,
 days) = months_days_list(folder_path, tickers[0], year)

with mp.Pool(processes=mp.cpu_count()) as pool:
    
    #for month in months_list:
        
    #    pool.starmap(taq_trade_signs_all_transactions_data,
    #                 product(tickers, [year], [month],
    #                 days[int(month) - 1]))
        
    for month in months_list:
        
        pool.starmap(taq_self_response_transactions_data,
                     product(tickers, [year], [month],
                     days[int(month) - 1]))
        pool.starmap(taq_cross_response_transactions_data,
                     product(tickers, tickers, [year], [month],
                     days[int(month) - 1]))

TAQ data
TAQ data
TAQ data
TAQ data
TAQ data
TAQ data
TAQ data
TAQ data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.01.16
Processing data for the stock AAPL the 2008.01.14
Processing data for the stock AAPL the 2008.01.04
Processing data for the stock AAPL the 2008.01.08
Processing data for the stock AAPL the 2008.01.02
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.01.23
Processing data for the stock AAPL the 2008.01.18
Processing data for the stock AAPL the 2008.01.10
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.01.15
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.01.03
Data Saved

TAQ data
taq_s

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.08
Data Saved
Data Saved


TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.30
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.11
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.22
Data Saved

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.16
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.25
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.01.31
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock


TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.02.05
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.02.25
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.02.08
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.02.06
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.02.13
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.02.19
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.02.22
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.0

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.24
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.03.05
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.13
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.05
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.10
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.18
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.03.28
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT

Data Saved

Data Saved

Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.04.24
Data Saved

Data Saved

Data Saved

Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.04.30
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.04.28
Data Saved

Data Saved

Data Saved

TAQ data
TAQ data
TAQ data
TAQ data
taq_cross_response_transactions_data
TAQ data
taq_cross_response_transactions_data
TAQ data
TAQ data
TAQ data
Processing data for the stock i AAPL and stock j MSFT the 2008.04.08
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.04.16
taq_cross_response_transactions_data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.04.01
taq_cross_response_transactions_data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stoc

...

Processing data for the stock MSFT the 2008.09.23
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.16
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.25
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.29
Data Saved

Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.18
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.22
Data Saved

Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.26
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.24
Data Saved

Data Saved

Data Saved

Data Saved

Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.09.30
Data Saved

Data Saved

Processing data for the stock AAPL the 2008.10.29
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.10.31
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.10.02
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.10.06
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.10.28
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.10.08
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock AAPL the 2008.10.24
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.10.10
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT the 2008.10.01
Data Saved

TAQ data
taq_self_response_transactions_data
Processing data for the stock MSFT th

Processing data for the stock i MSFT and stock j AAPL the 2008.10.30
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.10.20
Data Saved

Data Saved

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.10.28
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.10.23
Data Saved

Data Saved

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.10.31
Data Saved

Data Saved

Data Saved

TAQ data
TAQ data
TAQ data
TAQ data
TAQ data
taq_self_response_transactions_data
TAQ data
TAQ data
Processing data for the stock AAPL the 2008.11.11
TAQ data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
taq_self_response_transactions_data
t


TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.11.18
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.11.25
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.11.17
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.11.25
Data Saved

Data Saved

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.11.19
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.11.21
Data Saved

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.11.26
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stoc

Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.12.09
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.12.23
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.12.05
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT the 2008.12.15
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.12.10
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.12.03
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i MSFT and stock j AAPL the 2008.12.15
Data Saved

TAQ data
taq_cross_response_transactions_data
Processing data for the stock i AAPL and stock j MSFT